In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from global_import import *
from valuation.val import perform_valuation

In [2]:
comp = pd.read_csv(f'{DATA_FOLDER}/comp_1959-2019.csv', index_col=0)
crsp = pd.read_csv(f'{DATA_FOLDER}/crsp_1959-2019.csv', index_col=0)
beta = pd.read_csv(f'{DATA_FOLDER}/capm_beta_weekly_1-3y_1959-2019.csv', index_col=0)

comp['jdate'] = pd.to_datetime(comp['jdate'])
comp.permno = comp.permno.astype(int)

beta['date']=pd.to_datetime(beta['DATE'].astype(str))
beta['jdate']=beta['date']+MonthEnd(0)
beta['permno'] = beta.index
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
FF = pd.read_csv(f'{PROJECT_FOLDER}/lookup_tables/FF3.csv')
FF['date'] = FF['Unnamed: 0']
FF['date'] = pd.to_datetime(FF['date'].astype(str), format='%Y%m%d')
FF['jdate'] = FF['date']+MonthEnd(0)
FF = FF[['jdate', 'date', 'RF']]
FF['RF'] = (1+FF['RF']/100).pow(365)-1

beta = beta.merge(FF[['jdate', 'RF']],  on=['jdate'])
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

In [4]:
comp.sort_values(['permno','year'], ascending = [True,False], inplace=True)
for i in range(11):
    comp[f'ni_{i}'] = comp.groupby('permno')['ni'].shift(i)
    comp[f'ni_{i}'] = comp[f'ni_{i}'].fillna(0)

In [5]:
comp.sort_values(['permno','jdate'], ascending = [True,True], inplace=True)
crsp['logret'] = np.log(1+crsp['retadj'])
crsp['termlogret'] = crsp.groupby('permno')['logret'].transform('sum') - crsp.groupby('permno')['logret'].transform('cumsum') + np.log(1-0.35) 
crsp['termcumret'] = np.exp(crsp['termlogret'])-1
crsp1 = crsp.copy()

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [6]:
crsp[['ticker', 'jdate', 'logret', 'termlogret']].head(3)

,ticker,jdate,logret,termlogret
0,OMFGA,1986-01-31,0.000000,-3.426515
1,OMFGA,1986-02-28,-0.297252,-3.129264
2,OMFGA,1986-03-31,0.311436,-3.440700


In [7]:
for x in ['logret_3', 'logret_6', 'logret_12', 'logret_24', 
      'logret_36', 'logret_48', 'logret_60']:
    crsp1.loc[crsp1[x].isnull(), x] = crsp1['termlogret']


for x in ['cumret_3', 'cumret_6', 'cumret_12', 'cumret_24', 
      'cumret_36', 'cumret_48', 'cumret_60']:
    crsp1.loc[crsp1[x].isnull(), x] = crsp1['termcumret']

In [8]:
valdf = comp.merge(beta, on=['jdate', 'permno'])
valdf = perform_valuation(valdf, n=5, rp=0.07, tg=0.05)
valdf = valdf[['permno', 'gvkey', 'year', 'PV', 'be']]
valdf.year = valdf.year + 1
val_crsp_mdf = valdf.merge(crsp1, on=['year', 'permno'])
val_crsp_mdf['vt'] = val_crsp_mdf['PV']*1000
val_crsp_mdf['be'] = val_crsp_mdf['be']*1000

In [9]:
val_df1 = val_crsp_mdf[['permno', 'ticker',  'exchcd', 'year', 'jdate', 'exchcd',
                        'vt','me', 'be', 'prc',
                        'logret_3', 'logret_6', 'logret_24', 'cumret_3', 'cumret_6', 'cumret_24',
                        'logret_12', 'logret_36', 'logret_48', 'logret_60', 
                        'cumret_12', 'cumret_36', 'cumret_48', 'cumret_60',]]
val_df1.sort_values(['permno','jdate'], ascending = [True,False], inplace=True)

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
val_df2 = val_df1.copy()
for m in [3, 6, 12, 24, 36, 48, 60]:
    val_df2[f'vt+{m}'] = val_df1.groupby('permno')['vt'].shift(m)
    val_df2[f'me+{m}'] = val_df1.groupby('permno')['me'].shift(m)
    val_df2[f'be+{m}'] = val_df1.groupby('permno')['be'].shift(m)

In [11]:
val_df3 = val_df2.copy()
for m in [3, 6, 12, 24, 36, 48, 60]:
    val_df3['vt'] = np.log(val_df2.vt)
    val_df3[f'vt+{m}'] = np.log(val_df2[f'vt+{m}'].astype(float))
    val_df3['me'] = np.log(val_df2['me'].astype(float))
    val_df3[f'me+{m}'] = np.log(val_df2[f'me+{m}'].astype(float))
    val_df3[f'r_t+{m}'] = val_df3[f'logret_{m}']
    val_df3[f'r*_t+{m}'] = val_df3[f'vt+{m}'] -  val_df3[f'vt']
    val_df3[f'r_unchg+{m}'] = - (1-0.975) * (val_df3['me'] -  val_df3['vt'])
    val_df3[f'r_chg+{m}'] = 0.975 * ( (val_df3[f'me+{m}'] -  val_df3[f'vt+{m}']) - ((val_df3[f'me'] -  val_df3['vt'])) )
    

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/mmw/Installations/venvs/mlval/lib/python3.7/s

In [12]:
val_df3 = val_df3[(val_df3.year >=1980) & (val_df3.year<=2014)]
val_df3 = val_df3.drop_duplicates(subset=['permno', 'jdate'])

In [13]:
val_df3['misp'] = val_df3['me'] - val_df3['vt']
val_df3 = val_df3.dropna(subset=['misp', 'r_t+60'])

In [14]:
val_df3[(val_df3.ticker=='AAPL') & (val_df3.ticker=='AAPL')]

,permno,ticker,exchcd,year,jdate,exchcd,vt,me,be,prc,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_48,logret_60,cumret_12,cumret_36,cumret_48,cumret_60,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+48,me+48,be+48,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+48,r*_t+48,r_unchg+48,r_chg+48,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
227695,14593,AAPL,3,2014,2014-12-31,3,21.294310,20.281842,140038000.0,110.379997,1.234664e-01,0.135415,0.086288,1.314120e-01,0.145011,0.090120,-0.030873,0.481271,0.425671,1.062266,-0.030401,0.618130,0.530617,1.892920,21.282744,20.401370,131806000.0,21.282744,20.388632,131806000.0,21.282744,20.184748,131806000.0,19.039989,20.227264,143417000.0,18.827261,20.573469,154268000.0,18.515270,20.430342,165551000.0,17.827487,20.989289,107573000.0,1.234664e-01,-0.011566,0.025312,0.127816,0.135415,-0.011566,0.025312,0.115396,-0.030873,-0.011566,0.025312,-0.083390,0.086288,-2.254321,0.025312,2.144749,0.481271,-2.467049,0.025312,2.689709,0.425671,-2.779040,0.025312,2.854351,1.062266,-3.466822,0.025312,4.069912,-1.012468
227694,14593,AAPL,3,2014,2014-11-30,3,21.294310,20.363021,140038000.0,118.930000,8.073458e-02,0.098786,-0.035159,8.408313e-02,0.103830,-0.034548,0.011311,0.422028,0.475154,0.893456,0.011375,0.525051,0.608261,1.443561,21.282744,20.433244,131806000.0,21.282744,20.436529,131806000.0,21.282744,20.307086,131806000.0,19.039989,20.194492,143417000.0,18.827261,20.598078,154268000.0,18.515270,20.557723,165551000.0,17.827487,20.895085,107573000.0,8.073458e-02,-0.011566,0.023282,0.079744,0.098786,-0.011566,0.023282,0.082947,0.011311,-0.011566,0.023282,-0.043260,-0.035159,-2.254321,0.023282,2.033647,0.422028,-2.467049,0.023282,2.634554,0.475154,-2.779040,0.023282,2.899398,0.893456,-3.466822,0.023282,3.898914,-0.931289
227693,14593,AAPL,3,2014,2014-10-31,3,21.294310,20.266617,140038000.0,108.000000,8.535344e-02,0.154978,0.087004,8.910193e-02,0.167632,0.090901,0.117366,0.502230,0.774818,0.919231,0.124531,0.652402,1.170196,1.507362,21.282744,20.341167,131806000.0,21.282744,20.396356,131806000.0,21.282744,20.317179,131806000.0,19.039989,20.221451,143417000.0,18.827261,20.581592,154268000.0,18.515270,20.761118,165551000.0,17.827487,20.823389,107573000.0,8.535344e-02,-0.011566,0.025692,0.083963,0.154978,-0.011566,0.025692,0.137772,0.117366,-0.011566,0.025692,0.060574,0.087004,-2.254321,0.025692,2.153926,0.502230,-2.467049,0.025692,2.712473,0.774818,-2.779040,0.025692,3.191702,0.919231,-3.466822,0.025692,3.923004,-1.027693
227692,14593,AAPL,3,2014,2014-09-30,3,21.294310,20.197353,140038000.0,100.750000,9.523085e-02,0.218697,0.152168,9.991274e-02,0.244454,0.164356,0.106742,0.479315,0.875258,0.883744,0.112648,0.614968,1.399495,1.419943,21.294310,20.281842,140038000.0,21.282744,20.401370,131806000.0,21.282744,20.259658,131806000.0,19.039989,20.227597,143417000.0,18.827261,20.487607,154268000.0,18.515270,20.794088,165551000.0,17.827487,20.718406,107573000.0,9.523085e-02,0.000000,0.027424,0.082377,0.218697,-0.011566,0.027424,0.210192,0.106742,-0.011566,0.027424,0.072024,0.152168,-2.254321,0.027424,2.227451,0.479315,-2.467049,0.027424,2.688370,0.875258,-2.779040,0.027424,3.291380,0.883744,-3.466822,0.027424,3.888178,-1.096957
227691,14593,AAPL,3,2014,2014-08-31,3,21.294310,20.235109,140038000.0,102.500000,1.526164e-01,0.233351,0.071499,1.648781e-01,0.262825,0.074117,0.111579,0.524230,0.866375,0.796101,0.118043,0.689157,1.378275,1.216879,21.294310,20.363021,140038000.0,21.282744,20.433244,131806000.0,21.282744,20.281716,131806000.0,19.039989,20.164149,143417000.0,18.827261,20.557326,154268000.0,18.515270,20.818063,165551000.0,17.827487,20.664931,107573000.0,1.526164e-01,0.000000,0.026480,0.124714,0.233351,-0.011566,0.026480,0.204458,0.111579,-0.011566,0.026480,0.056719,0.071499,-2.254321,0.026480,2.128777,0.52

In [15]:
val_df3 = val_df3.replace([np.inf, -np.inf], np.nan)

In [16]:
val_df3.to_csv(f'{DATA_FOLDER}/decomp_dcf5y_hist_1980_2014.csv')

In [17]:
val_df3.describe()

,permno,exchcd,year,exchcd,vt,me,be,prc,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_48,logret_60,cumret_12,cumret_36,cumret_48,cumret_60,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+48,me+48,be+48,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+48,r*_t+48,r_unchg+48,r_chg+48,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
count,1.241569e+06,1.241569e+06,1.241569e+06,1.241569e+06,1.027597e+06,1.241322e+06,1.093365e+06,1.241322e+06,1.241559e+06,1.241541e+06,1.241486e+06,1.241569e+06,1.241569e+06,1.241569e+06,1.241508e+06,1.241474e+06,1.241462e+06,1.241450e+06,1.241569e+06,1.241569e+06,1.241569e+06,1.241569e+06,982169.000000,1.209181e+06,1.066335e+06,937417.000000,1.177203e+06,1.039468e+06,847680.000000,1.122594e+06,9.934650e+05,761038.000000,1.052951e+06,9.347610e+05,696377.000000,990455.000000,8.807200e+05,646148.000000,932610.000000,8.299800e+05,591652.000000,879471.000000,7.849900e+05,1.241559e+06,981972.000000,1.027350e+06,981574.000000,1.241541e+06,937135.000000,1.027350e+06,936767.000000,1.241508e+06,847116.000000,1.027350e+06,846829.000000,1.241486e+06,760042.000000,1.027350e+06,759795.000000,1.241474e+06,694996.000000,1.027350e+06,694792.000000,1.241462e+06,644396.000000,1.027350e+06,644224.000000,1.241450e+06,589613.000000,1.027350e+06,589466.000000,1.027350e+06
mean,5.632803e+04,1.971681e+00,1.997406e+03,1.971681e+00,1.241536e+01,1.234944e+01,1.575814e+06,3.488406e+01,1.242635e-02,2.292409e-02,1.061937e-01,3.941945e-02,7.807216e-02,3.147401e-01,4.639834e-02,1.746742e-01,2.333603e-01,2.731061e-01,1.574106e-01,4.954714e-01,6.727451e-01,8.484054e-01,12.480372,1.240827e+01,1.633469e+06,12.550588,1.247049e+01,1.693835e+06,12.713458,1.258126e+01,1.812161e+06,12.881520,1.271638e+01,2.006533e+06,13.008404,12.856294,2.213757e+06,13.098149,12.989499,2.448606e+06,13.244151,13.100125,2.718019e+06,1.242635e-02,-0.001868,-2.842209e-03,0.028882,2.292409e-02,-0.003731,-2.842209e-03,0.059396,4.639834e-02,-0.008046,-2.842209e-03,0.124321,1.061937e-01,-0.039691,-2.842209e-03,0.265367,1.746742e-01,-0.080986,-2.842209e-03,0.422915,2.333603e-01,-0.143381,-2.842209e-03,0.589878,2.731061e-01,-0.085453,-2.842209e-03,0.636752,1.136884e-01
std,2.575933e+04,9.433011e-01,9.724783e+00,9.433011e-01,2.840529e+00,2.200510e+00,7.093923e+06,9.568488e+02,2.353023e-01,3.386345e-01,6.132119e-01,2.530283e-01,3.839663e-01,9.591962e-01,4.720338e-01,7.103240e-01,7.867679e-01,8.664685e-01,8.619777e-01,1.691351e+00,2.331723e+00,3.378108e+00,2.831057,2.191122e+00,7.286266e+06,2.818852,2.178044e+00,7.482051e+06,2.783780,2.152120e+00,7.859416e+06,2.762776,2.149914e+00,8.497900e+06,2.746772,2.148198,9.159633e+06,2.731081,2.151570,9.892351e+06,2.680432,2.164640,1.067917e+07,2.353023e-01,0.471945,4.540470e-02,0.512282,3.386345e-01,0.680595,4.540470e-02,0.734469,4.720338e-01,1.012154,4.540470e-02,1.082523,6.132119e-01,1.360812,4.540470e-02,1.454705,7.103240e-01,1.612186,4.540470e-02,1.731502,7.867679e-01,1.832716,4.540470e-02,1.951164,8.664685e-01,1.950666,4.540470e-02,2.042863,1.816188e+00
min,1.000100e+04,1.000000e+00,1.980000e+03,1.000000e+00,1.835761e-01,1.963610e+00,1.000000e+00,1.562500e-02,-9.478570e+00,-9.780850e+00,-1.042955e+01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.042955e+01,-1.042955e+01,-1.042955e+01,-1.165372e+01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.183576,1.963610e+00,1.000000e+00,0.183576,2.117760e+00,1.000000e+00,0.693147,3.279500e+00,1.000000e+00,0.183576,2.888634e+00,1.000000e+00,0.183576,3.518980,1.400000e+01,0.183576,1.571217,6.000000e+00,0.183576,1.424613,1.400000e+01,-9.478570e+00,-10.075663,-3.612986e-01,-8.615118,-9.780850e+00,-10.075663,-3.612986e-01,-9.239290,-1.042955e+01,-10.075663,-3.612986e-01,-9.603316,-1.042955e+01,-14.098419,-3.612986e-01,-10.639336,-1.042955e+01,-14.140038,-3.612986e-01,-11.416517